In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm


  # graph it
        

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [0]:
path = os.path.abspath('.cnn.py')

In [0]:
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file

In [6]:
print(path)

/content/


In [0]:
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')

In [8]:
img_rows = 256
img_cols = 256
num_class = 21
path = os.path.abspath('.cnn.py') #absolute path of program
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
x_train = []
y_train = []
x_test = []
y_test = []
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')
dirs=dirs[:-1]
print(len(dirs))
'''

'''

20


'\n\n'

In [0]:
label = 0
for i in dirs:
	n = 0
	count = 0
	for pic in glob.glob(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'+i+'/*.tif'):
		im = Image.open(pic)
		im = np.array(im)
		if((im.shape[0]==256) and (im.shape[1] ==256) and count <90): #get only 90 data
			r = im[:,:,0]
			g = im[:,:,1]
			b = im[:,:,2]
			if(n<5): # 5 data in beginning set as test data
				x_test.append([r,g,b])
				y_test.append([label])
			else: #remaining data set as training data
				x_train.append([r,g,b])
				y_train.append([label])
			n = n + 1
			count = count + 1
		#print(count)
	label = label + 1

In [0]:
x_trainbck=x_train
x_testbck=x_test


In [0]:
x_train=np.array(x_train)
x_test=np.array(x_test)

In [0]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)

input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, 21)
y_test = keras.utils.to_categorical(y_test, 21)

In [13]:
!pip install  split_folders tqdm

In [14]:
import split_folders
split_folders.ratio(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/', output=path+'/drive/My Drive/ucmerced/UCMerced_LandUse/output/', seed=1337, ratio=(.8, .2))


Copying files: 2100 files [01:22,  2.33 files/s]


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import time
IMG_SIZE =256 # Replace with the size of your images
NB_CHANNELS =3 # 3 for RGB images or 1 for grayscale images
BATCH_SIZE = 32# Typical values are 8, 16 or 32
NB_TRAIN_IMG = 1680# Replace with the total number training images
NB_VALID_IMG = 420# Replace with the total number validation images

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,NB_CHANNELS),
               data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [18]:
train_datagen = ImageDataGenerator(
    rotation_range = 40,                  
    width_shift_range = 0.2,                  
    height_shift_range = 0.2,                  
    rescale = 1./255,                  
    shear_range = 0.2,                  
    zoom_range = 0.2,                     
    horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(
    path+'/drive/My Drive/ucmerced/UCMerced_LandUse/output/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size = BATCH_SIZE)
validation_generator = validation_datagen.flow_from_directory(
    path+'/drive/My Drive/ucmerced/UCMerced_LandUse/output/val',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size = BATCH_SIZE)

Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.


In [19]:
start = time.time()
model.fit_generator(
    train_generator,
    steps_per_epoch=NB_TRAIN_IMG//BATCH_SIZE,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=NB_VALID_IMG//BATCH_SIZE)
end = time.time()
print('Processing time:',(end - start)/60)
model.save_weights('cnn_baseline.h5')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





52/52 [==============================] - 33s 640ms/step - loss: 3.1038 - acc: 0.0637 - val_loss: 2.9065 - val_acc: 0.0601
Epoch 2/50
52/52 [==============================] - 31s 594ms/step - loss: 2.9053 - acc: 0.1004 - val_loss: 2.7224 - val_acc: 0.1495
Epoch 3/50
52/52 [==============================] - 31s 593ms/step - loss: 2.8100 - acc: 0.1502 - val_loss: 2.5999 - val_acc: 0.1959
Epoch 4/50
52/52 [==============================] - 30s 584ms/step - loss: 2.6520 - acc: 0.1815 - val_loss: 2.4486 - val_acc: 0.2010
Epoch 5/50
52/52 [==============================] - 31s 587ms/step - loss: 2.6208 - acc: 0.2073 - val_loss: 2.5721 - val_acc: 0.2629
Epoch 6/50
52/52 [==============================] - 31s 589ms/step - loss: 2.4895 - acc: 0.2314 - val_loss: 2.1850 - val_acc: 0.2938
Epoch 7/50
52/52 [==============================] - 30s 583ms/step - loss: 2.4111 - acc: 0.2488 - va